In [ ]:
import sys
sys.path.append('../..')

from src.index_files import *

In [ ]:
dataset = QualityDataset(split='dev')
f = Factory(chunk_size=100, llm_name=None)

In [ ]:
article = dataset.get_article(dataset.data[2])

In [ ]:
import spacy
from rank_bm25 import BM25Okapi
import spacy.tokens

class ChunkInfo(BaseModel):
    i: int
    chunk_text: str
    statements: List[str] = []
    entities: List[List[str]] = []
    keywords: List[List[str]] = []
        

class LongDoc:
    
    def __init__(self, factory:Factory, chunk_info_file:str=None) -> None:
        self.factory = factory
        self.nlp = spacy.load('en_core_web_lg')
        if chunk_info_file:
            self.chunk_infos = [ChunkInfo.parse_obj(line) for line in read_json(chunk_info_file)]
            self.build_relation_graph()
            self.build_lexical_store()
        
    def build_index(self, article:str, chunk_info_file:str=None):
        pieces = self.factory.split_text(article)
        print('generate_statements')
        chunks, statements = self.generate_statements(pieces)
        # statements = [ci.statements for ci in chunk_infos]
        print('end')
        chunk_infos = [ChunkInfo(i=i, chunk_text=chunk) for i, chunk in enumerate(chunks)]
        missing_chunk_ids = [ci.i for ci in chunk_infos if not ci.statements]
        attempt = 0
        while missing_chunk_ids:
            print(attempt, len(missing_chunk_ids))
            attempt += 1
            if len(missing_chunk_ids) == len(statements):
                entities = self.extract_entities(statements)
                for cid, (statement_list, entity_list) in enumerate(zip(statements, entities)):
                    if len(statement_list) == len(entity_list):
                        chunk_infos[cid].statements, chunk_infos[cid].entities = statement_list, entity_list
            else:
                temp_stm_groups = []
                for missing_ci in missing_chunk_ids:
                    temp_stms = statements[missing_ci]
                    split = np.random.randint(len(statements) // 3, len(statements) * 2 // 3)
                    temp_stm_groups.append(temp_stms[:split])
                    temp_stm_groups.append(temp_stms[split:])
                entities = self.extract_entities(temp_stm_groups)
                for sid in range(len(temp_stm_groups)//2):
                    if len(temp_stm_groups[sid * 2]) == len(entities[sid * 2]) and len(temp_stm_groups[sid * 2 + 1]) == len(entities[sid * 2 + 1]):
                        chunk_infos[missing_chunk_ids[sid]].statements, chunk_infos[missing_chunk_ids[sid]].entities = temp_stm_groups[sid * 2] + temp_stm_groups[sid * 2 + 1], entities[sid * 2] + entities[sid * 2 + 1]
            missing_chunk_ids = [ci.i for ci in chunk_infos if not ci.statements]

        for ci in chunk_infos:
            for sid, statement in enumerate(ci.statements):
                addition_ents, keywords = self.collect_keywords_from_text(statement)
                ci.keywords.append(keywords)
                for addition_ent in addition_ents:
                    if all([addition_ent.lower() not in ent.lower() for ent in ci.entities[sid]]):
                        ci.entities[sid].append(addition_ent)
        
        self.chunk_infos = chunk_infos
        self.build_relation_graph()
        self.build_lexical_store()
        if chunk_info_file:
            write_json(chunk_info_file, [ci.dict() for ci in chunk_infos])

    def collect_keywords_from_text(self, text:str):
        
        def trim_det_adv(noun_chunk:spacy.tokens.Span):
            for tid, t in enumerate(noun_chunk):
                if t.pos_ not in ['DET', 'ADV']:
                    return noun_chunk[tid:]
                
        doc = self.nlp(text)
        ncs = [trim_det_adv(nc) for nc in doc.noun_chunks if nc.root.pos_ not in ['NUM', 'PRON']]
        ents = [trim_det_adv(ent) for ent in doc.ents if ent.root.pos_ not in ['NUM', 'PRON']]   
        keywords = [(t.i, t.i+1) for t in doc if t.pos_ in ['VERB', 'ADJ', 'ADV']]
        ncs_spans = [(nc.start, nc.end) for nc in ncs]
        ents_spans = [(ent.start, ent.end) for ent in ents]
        nc_id, eid = 0, 0
        spans = []
        while nc_id < len(ncs_spans) and eid < len(ents_spans):
            nc_span, ent_span = ncs_spans[nc_id], ents_spans[eid]
            if set(range(*nc_span)).intersection(range(*ent_span)):
                merged_span = (min(nc_span[0], ent_span[0]), max(nc_span[1], ent_span[1]))
                spans.append(merged_span)
                nc_id += 1
                eid += 1
            else:
                if nc_span[0] < ent_span[0]:
                    spans.append(nc_span)
                    nc_id += 1
                else:
                    spans.append(ent_span)
                    eid += 1
        spans.extend(ncs_spans[nc_id:])
        spans.extend(ents_spans[eid:])
        updated_spans:List[Tuple[int, int]] = []
        for span in spans:
            doc_span = doc[span[0]:span[1]]
            if ',' in doc_span.text:
                start = doc_span.start
                for t in doc_span:
                    if t.text == ',':
                        if t.i != start:
                            updated_spans.append((start, t.i))
                        start = t.i + 1
                if start < span[1]:
                    updated_spans.append((start, span[1]))
            else:
                updated_spans.append(span)
        updated_spans = [span for span in updated_spans if any([t.pos_ in ['NOUN', 'PROPN'] for t in doc[span[0]:span[1]]])]
        updated_spans = [span if doc[span[0]].pos_ != 'PRON' else (span[0]+1, span[1]) for span in updated_spans]
        ent_candidates = [doc[span[0]:span[1]].text for span in updated_spans]
        ent_candidates = [ent.strip('"') for ent in ent_candidates]
        ent_candidates = [ent for ent in ent_candidates if len(ent) >= 2]
        
        keywords = updated_spans + keywords
        keywords.sort(key=lambda x: x[0])
        kw_candidates = [doc[span[0]:span[1]].text for span in keywords]
        kw_candidates = [ent.strip('"') for ent in kw_candidates]
        return ent_candidates, kw_candidates
    
    def parse_statements(self, text:str):
        i = 1
        statements:List[str] = []
        for line in text.strip().splitlines():
            if line.startswith(f'{i}. '):
                statements.append(line.split(' ', 1)[1].strip())
                i += 1
        return statements

    def parse_entities(self, text:str):
        i = 1
        list_of_ent_list:List[List[str]] = []
        for line in text.strip().splitlines():
            line = line.strip()
            if line.startswith(f'{i}. '):
                temp_ent_list = line.split(' ', 1)[1].strip().split(',')
                ent_list:List[str] = []
                incomplete_ent = []
                for ent in temp_ent_list:
                    if '(' in ent and ')' not in ent:
                        incomplete_ent.append(ent)
                    elif '(' not in ent and ')' in ent:
                        incomplete_ent.append(ent)
                        ent_list.append(','.join(incomplete_ent).strip().strip('.'))
                        incomplete_ent.clear()
                    elif incomplete_ent:
                        incomplete_ent.append(ent)
                    else:
                        ent_list.append(ent.strip().strip('.'))
                ent_list = [ent.split(':', 1)[1].strip() if ent.startswith('Entities:') else ent for ent in ent_list]
                ent_list = [self.clean_entity(ent) for ent in ent_list]
                ent_list = [ent for ent in ent_list if ent]
                list_of_ent_list.append(ent_list)
                i += 1
        return list_of_ent_list
    
    def build_relation_graph(self):
        relation_graph = nx.Graph()
        # semantic edges
        for ci in self.chunk_infos:
            for sid, related_ents in enumerate(ci.entities):
                loc = (ci.i, sid)
                # Insert node locs
                for e in related_ents:
                    if not relation_graph.has_node(e):
                        relation_graph.add_node(e, locs=[], norm=' '.join(self.normalize_entity(e)))
                    ent_locs:list = relation_graph.nodes[e]['locs']
                    if loc not in ent_locs:
                        ent_locs.insert(0, loc)
                for ent1, ent2 in itertools.combinations(related_ents, 2):
                    if not relation_graph.has_edge(ent1, ent2):
                        relation_graph.add_edge(ent1, ent2, locs=[])
                    edge_locs:list = relation_graph[ent1][ent2]['locs']
                    edge_locs.append((loc, loc))
        
        self.normal2ents:Dict[str, List[str]] = defaultdict(list)
        for ent, normal in relation_graph.nodes(data='norm'):
            self.normal2ents[normal].append(ent)
        normals = list(self.normal2ents)
        normals.sort()
        self.ent_corpus = [ent.split() for ent in normals]
        self.ent_bm25 = BM25Okapi(self.ent_corpus)
        
        for normal in self.normal2ents:
            # Add edges between entities that have the same norm
            for ent1, ent2 in itertools.combinations(self.normal2ents[normal], 2):
                if not relation_graph.has_edge(ent1, ent2):
                    relation_graph.add_edge(ent1, ent2, locs=[])
                edge_locs:list = relation_graph[ent1][ent2]['locs']
                edge_locs.extend(itertools.product(relation_graph.nodes[ent1]['locs'], relation_graph.nodes[ent2]['locs']))
            # Add edges between entities that have similar norms
            scores:List[float] = self.ent_bm25.get_scores(normal.split()).tolist()
            for score, similar_normal in zip(scores, self.ent_corpus):
                if score > 0:
                    similar_normal = ' '.join(similar_normal)
                    if normal != similar_normal:
                        for ent1, ent2 in itertools.product(self.normal2ents[normal], self.normal2ents[similar_normal]):
                            if not relation_graph.has_edge(ent1, ent2):
                                relation_graph.add_edge(ent1, ent2, locs=[])
                            edge_locs:list = relation_graph[ent1][ent2]['locs']
                            edge_locs.extend(itertools.product(relation_graph.nodes[ent1]['locs'], relation_graph.nodes[ent2]['locs']))
        
        self.graph = relation_graph
    
    def build_lexical_store(self):
        self.raw_corpus = [self.normalize_text(ci.chunk_text) for ci in self.chunk_infos]
        self.raw_bm25 = BM25Okapi(self.raw_corpus)
        
    def lexical_retrieval_chunks(self, query:str, n:int=5):
        chunk_idxs = self.bm25_retrieve(self.normalize_text(query), self.raw_bm25)
        return [(self.chunk_infos[idx].chunk_text, idx) for idx in chunk_idxs][:n]
    
    def lexical_retrieval_entities(self, query:str, n:int=5):
        tokenized_query = self.normalize_entity(query)
        normal_idxs = self.bm25_retrieve(tokenized_query, self.ent_bm25)
        candidate_normals = [' '.join(self.ent_corpus[idx]) for idx in normal_idxs]
        temp_ents = []
        for normal in candidate_normals:
            temp_ents.extend(self.normal2ents[normal])
        temp_ent_corpus = [self.split_lower_text(ent) for ent in temp_ents]
        temp_bm25 = BM25Okapi(temp_ent_corpus)
        ent_idxs = self.bm25_retrieve(tokenized_query, temp_bm25)
        return [temp_ents[idx] for idx in ent_idxs][:n]
    
    def exact_match_chunks(self, query:str):
        normalized_query = ' '.join(self.normalize_text(query))
        return [(self.chunk_infos[idx].chunk_text, idx) for idx, normalized_chunk in enumerate(self.raw_corpus) if normalized_query in ' '.join(normalized_chunk)]
        
    def generate_statements(self, pieces:List[str], chunk_size:int=5, summary_size:int=25, overlap:int=1):
        summary_chunks = concate_with_overlap(pieces, summary_size, overlap=overlap)
        summaries = self.factory.llm.generate([[HumanMessage(content=summary_prompt.format(chunk=' '.join(summary_chunk)))] for summary_chunk in summary_chunks])
        prompts = []
        chunks:List[str] = []
        for summary_chunk, summary in zip(summary_chunks, summaries.generations):
            temp_summary_size = min(summary_size, len(summary_chunk))
            summary_chunk = summary_chunk[:temp_summary_size]
            for batch_start in range((temp_summary_size + 1) // chunk_size):
                chunk = ' '.join(summary_chunk[batch_start * chunk_size : (batch_start + 1) * chunk_size])
                prompts.append(statement_prompt.format(summary=summary[0].text, chunk=chunk))
                chunks.append(chunk)
        return chunks, [self.parse_statements(gen[0].text) for gen in self.factory.llm.generate([[HumanMessage(content=prompt)] for prompt in prompts]).generations]
        
    def clean_entity(self, ent_text:str):
        ent_doc = self.nlp(ent_text, disable=['parser', 'ner'])
        for tid, t in enumerate(ent_doc):
            if t.pos_ not in ['DET', 'CCONJ', 'PRON']:
                return ent_doc[tid:].text
            
    def normalize_entity(self, ent_text:str):
        # return [t.text.lower() if t.pos_ != 'NOUN' else t.lemma_.lower() for t in self.nlp(ent_text, disable=['parser', 'ner']) if t.pos_ not in ['DET', 'PUNCT', 'ADP', 'SCONJ', 'PRON', 'CCONJ', 'PART', 'AUX']]
        return [t.text.lower() if t.pos_ != 'NOUN' else t.lemma_.lower() for t in self.nlp(ent_text, disable=['parser', 'ner']) if not (t.is_stop or t.pos_ == "PUNCT")]
    
    def normalize_text(self, text:str):
        return [t.lemma_.lower() if t.pos_ in ['NOUN', 'VERB'] else t.text.lower() for t in self.nlp(text, disable=['ner', 'parser']) if not t.is_stop]
    
    def split_lower_text(self, text:str) -> List[str]:
        return [t.text.lower() for t in self.nlp(text, disable=['ner', 'parser'])]
    
    def bm25_retrieve(self, tokenized_query:List[str], bm25:BM25Okapi):
        index_score_pairs = [(idx, score) for idx, score in enumerate(bm25.get_scores(tokenized_query)) if score > 0]
        index_score_pairs.sort(key=lambda x: x[1], reverse=True)
        return [idx for idx, _ in index_score_pairs]
        
    def extract_entities(self, statements:List[List[str]]):
        return [self.parse_entities(gen[0].text) for gen in self.factory.llm.generate([[HumanMessage(content='List the entities in each line of the following statements.\n\nStatements:\n' + '\n'.join([f'{sid+1}. {s}' for sid, s in enumerate(statement)]))] for statement in statements]).generations]


longdoc = LongDoc(f, 'atomic_facts.json')

In [ ]:
questions, answers = dataset.get_questions_and_answers(dataset.data[2])
questions

## Question Examples

### Example 1

Why does the Skipper stop abruptly after he says "when you\'re running a blockade"?

In [ ]:
longdoc.exact_match_chunks('''when you\'re running a blockade''')

In [ ]:
longdoc.exact_match_chunks('''stop abruptly''')

In [ ]:
longdoc.lexical_retrieval_entities('Skipper', 20)

### Example 2

Why does the Skipper allow the new chef to use the heat-cannon as an incinerator?

In [ ]:
longdoc.lexical_retrieval_entities('heat-cannon', 10)

In [ ]:
longdoc.lexical_retrieval_entities('incinerator')

In [ ]:
longdoc.lexical_retrieval_entities('skipper', 20)

In [ ]:
longdoc.lexical_retrieval_entities('new chef', 20)

In [ ]:
info_collection = defaultdict(Counter)
target_ents = ['heat-cannon', 'incinerator', 'skipper', 'new chef']
for target_ent in target_ents:
    for real_ent in longdoc.lexical_retrieval_entities(target_ent, 20):
        if (target_ent, real_ent) in [('new chef', 'new incinerator shipshape'), ('new chef', 'new incinerator (the old Nolan heat cannon)')]:
            continue
        for pid, sid in longdoc.graph.nodes[real_ent]['locs']:
            info_collection[pid][target_ent] += 1
df = pd.DataFrame({target_ent: [info_collection[i][target_ent] for i in range(len(longdoc.chunk_infos))] for target_ent in target_ents}, index=range(len(longdoc.chunk_infos)))
df.plot(kind='bar', stacked=True)
plt.xlabel('Chunks')
plt.ylabel('Entity occurrence')

In [ ]:
longdoc.chunk_infos[6].statements

In [ ]:
longdoc.chunk_infos[5].chunk_text

In [ ]:
longdoc.graph.nodes['new cook']

In [ ]:
longdoc.graph.nodes

In [ ]:
pr = nx.pagerank(longdoc.graph)

In [ ]:
pr

In [ ]:
sorted(list(pr.items()), key=lambda x: x[1], reverse=True)

In [ ]:
list(longdoc.graph.neighbors('Spaceship Leo'))

In [ ]:
longdoc.graph['Spaceship Leo']['Leo (spaceship)']

In [ ]:
[t.pos_ for t in longdoc.nlp("he is good at it")]

In [ ]:
a = f.llm.generate([[HumanMessage(content='''Below is a list of entity mentions. Group the mentions that refer to the same entity and give an entity name for each group.\n\nEntity mentions:\n''' + '\n'.join(ent_set) + '''\n\nIf there are duplicated mentions, generate your response in the following format:\n1. Entity 1's name: list of mentions\n2. Entity 2's name: list of mentions\n...\n\nIf there is no duplicate, simply generate "None".''')]])

In [ ]:
print(a.generations[0][0].text)

In [ ]:
class_ent

In [ ]:
json.loads(chunk_infos[1].json())

In [ ]:
longdoc.collect_keywords_from_text(chunk_infos[1].statements[3])

In [ ]:
vectorstore = Chroma.from_texts(texts=pieces, embedding=f.embeder)
# retriever = vectorstore.similarity_search_with_relevance_scores(pieces[0])

In [ ]:
def cal_compression(source_text:str, output_text:str):
    return float(len(word_tokenize(output_text))) / len(word_tokenize(source_text))

for chunk_size in tqdm((5, 10, 15, 20)):
    chunks = concate_with_overlap(pieces, chunk_size=chunk_size)
    results = f.llm.generate([[HumanMessage(content=f'Rewrite the following passage into a list of statements.\nEach statement should tell an atomic fact in the passage.\nAll the statements together should cover all the information in the passage.\nTry to use the original words from the passage.\n\nPassage:\n{chunk}')] for chunk in chunks])
    compression_ratios = []
    for cid, (output, source) in enumerate(zip(results.generations, chunks)):
        if cid != len(chunks) - 1:
            compression_ratios.append(cal_compression(source, output[0].text))
    print(chunk_size, np.mean(compression_ratios))

In [ ]:
from gensim import corpora
from gensim.parsing.preprocessing import preprocess_string, DEFAULT_FILTERS
from gensim.models import Phrases, CoherenceModel, LdaModel, EnsembleLda, LdaMulticore

In [ ]:
preprocess_funcs = DEFAULT_FILTERS[:-1] # Remove the stemming
preprocessed_summary = [preprocess_string(' '.join([t.lemma_ for t in longdoc.nlp(p, disable=['parser', 'ner'])]), preprocess_funcs) for p in all_summary]

# bigram = Phrases(preprocessed_summary, min_count=2, threshold=1)

# texts = [bigram[p] for p in preprocessed_summary]
texts = preprocessed_summary

# Create a dictionary from the corpus
dictionary = corpora.Dictionary(texts)

# Remove low-frequency terms from the dictionary
dictionary.filter_extremes(no_below=2)

# Convert the corpus into a bag-of-words representation
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
dataset.get_questions_and_answers(dataset.data[5])

In [ ]:
eli5 = load_dataset('defunct-datasets/eli5')

In [ ]:
!wget https://github.com/nyu-mll/SQuALITY/blob/main/data/v1-3/txt/dev.jsonl

In [ ]:
with open('dev.jsonl') as f_in:
    squality = f_in.read()
    # squality = [json.loads(l) for l in f_in]

In [ ]:
squality[:100]

In [ ]:
squality = load_dataset('pszemraj/SQuALITY-v1.3')